In [ ]:
import pandas as pd # pandas : 데이터 처리, 가공 및 분석을 편리하게 함
import numpy as np # numpy : 행렬이나 일반적으로 대규모 다차원 배열을 쉽게 처리
import matplotlib.pyplot as plt # matplotlib의 pyplot을 plt라는 이름으로 import
# matplotlib : 자료를 시각화하는데 사용하는 대표적인 라이브러리
import matplotlib.image as mpimg # matplotlib의 image를 mpimg라는 이름으로 import
import seaborn as sns # seaborn : matplotlib 기반의 파이썬 데이터 시각화 라이브러리

%matplotlib inline
# notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것
np.random.seed(2) # seed 설정 # seed : 특정 시작 숫자

# 훈련데이터 세트 분할
from sklearn.model_selection import train_test_split
# sklearn.model_selection 모듈에서train_test_split() 함수를 import
from sklearn.metrics import confusion_matrix # 혼동행렬을 위한 기능 불러옴
import itertools # 자신만의 반복자를 만드는 모듈

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
# one-hot encoding : 1차원 배열에 각각의 숫자일 확률을 기록 & 그 확률이 가장 높은 것을 정답으로 취급
from keras.models import Sequential # Sequential 모델 : 레이어를 선형으로 연결하여 구성
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop # RMSprop : 딥러닝 최적화 기법
from keras.preprocessing.image import ImageDataGenerator
# 실시간 데이터 증강을 사용해서 텐서 이미지 데이터 배치를 생성
from keras.callbacks import ReduceLROnPlateau # 콜백함수
# 학습률이 개선되지 않을 때, 학습률을 동적으로 조정하여 학습률을 개선하는 효과


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
from __future__ import print_function
# __future__ : 파이썬2.x 에서 몇몇 기능들을 파이썬3.x 와 같이 사용 가능하게 만들어줌
import keras
from keras.datasets import mnist # mnist 데이터셋 호출
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train : 2828의 행렬을 2828 픽셀로 생각했을때, 숫자가 작을수록 해당 픽셀의 색이
#진한 검정색을 의미. (0에 가까울수록 검은색, 255에 가까울수록 하얀색)
#y_train : 정답(숫자)

if K.image_data_format() == 'channels_first':
  # image_data_format이 ‘channels_first’인 경우 (샘플 수, 채널 수, 행, 열)로 이루어진 4D 텐서
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# 학습에 적합한 형태로 데이터 가공
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
# ((60000, 28, 28), (60000, ))
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 클래스를 학습에 이용하기 위해 데이터 가공
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# 딥러닝 모델 구조 설정
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

# (2개층, 128개의 뉴런 연결, 10개 클래스 출력 뉴런, relu와 softmax 활성화 함수 이용)
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# 딥러닝 구조 설정(loss 옵션을 다중 클래스에 적합한 categorical_crossentropy, 옵티마이저는 Adadelta 설정)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

# 학습 loss값, 정확도 출력
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# 모델 저장
model.save('digit_model.h5')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 155s 328ms/step - loss: 2.2794 - accuracy: 0.1495 - val_loss: 2.2261 - val_accuracy: 0.3391
Epoch 2/10
469/469 [==============================] - 151s 321ms/step - loss: 2.1965 - accuracy: 0.2764 - val_loss: 2.1257 - val_accuracy: 0.5381
Epoch 3/10
469/469 [==============================] - 153s 327ms/step - loss: 2.0957 - accuracy: 0.3783 - val_loss: 1.9962 - val_accuracy: 0.6642
Epoch 4/10
469/469 [==============================] - 151s 321ms/step - loss: 1.9633 - accuracy: 0.4620 - val_loss: 1.8287 - val_accuracy: 0.7266
Epoch 5/10
469/469 [==============================] - 153s 326ms/step - loss: 1.8006 - accuracy: 0.5333 - val_loss: 1.6266 - val_accuracy: 0.7590
Epoch 6/10
469/469 [==============================] - 153s 326ms/step - loss: 1.6233 - accuracy: 0.5760 - val_loss: 1.4144 - val_accuracy: 0.7740
Epoch 7/10
469/469 [==============================]

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras "digit_model.h5" "./model"

2023-01-10 09:23:43.149560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-10 09:23:43.149808: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-10 09:23:43.149833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
# import the necessary libraries
from keras.models import load_model
from flask import Response
from flask import request
from flask import Flask
import numpy as np
import base64
import cv2


app = Flask(__name__) # Flask 인스터스 생성 
model = None

@app.route('/predict', methods=['POST'])
def predict():
    content = request.form['data']
    img = np.fromstring(base64.b64decode(content[22:]), dtype=np.uint8)
    character = cv2.imdecode(img, 0)
    resized_character = cv2.resize(character, (28, 28)).astype('float32') / 255
    number = model.predict_classes(resized_character.reshape((1, 784)))[0]
    resp = Response(str(number))
    resp.headers['Access-Control-Allow-Origin'] = '*'
    return resp


@app.route('/sample_post', methods=['POST'])
def sample_post():
    content = request.form['data']
    return content


@app.route('/')
def hello_world():
    character = cv2.imread('3.png', 0)
    resized_character = cv2.resize(character, (28, 28)).astype('float32') / 255
    number = model.predict_classes(resized_character.reshape((1, 28 * 28)))[0]
    resp = Response(str(number))
    resp.headers['Access-Control-Allow-Origin'] = '*'
    return resp

if __name__ == '__main__':
    model = load_model('digit_model.h5')
    app.run(debug=True, host='0.0.0.0', port=8888)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://0.0.0.0:8888/ (Press CTRL+C to quit)
INFO:werkzeug: * Restarting with stat


In [ ]:
from google.colab import drive
drive.mount('/content/drive')